In [ ]:
import json
# read api key from tally_api_key.json
TALLY_API_KEY = json.load(open('tally_api_key.json'))['key']

In [ ]:
import requests

url = "https://api.tally.xyz/query"

headers = {
  'Api-Key': TALLY_API_KEY,
  'Content-Type': 'application/json'
}

# Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive_path = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/"

# Chains

In [ ]:
payload="{\"query\":\"query Chains {\\n    chains {\\n        id\\n    }\\n}\",\"variables\":{}}"

chains_response = requests.request("POST", url, headers=headers, data=payload)

chains_data = chains_response.json()
chains_data

{'data': {'chains': [{'id': 'eip155:69'},
   {'id': 'eip155:46'},
   {'id': 'eip155:300'},
   {'id': 'eip155:1480'},
   {'id': 'eip155:534353'},
   {'id': 'eip155:421613'},
   {'id': 'eip155:69000'},
   {'id': 'eip155:84531'},
   {'id': 'eip155:42161'},
   {'id': 'eip155:14800'},
   {'id': 'eip155:89'},
   {'id': 'eip155:5'},
   {'id': 'eip155:59140'},
   {'id': 'eip155:314159'},
   {'id': 'eip155:80001'},
   {'id': 'eip155:7560'},
   {'id': 'eip155:130'},
   {'id': 'eip155:42'},
   {'id': 'eip155:44'},
   {'id': 'eip155:369'},
   {'id': 'eip155:133'},
   {'id': 'eip155:255'},
   {'id': 'eip155:43113'},
   {'id': 'eip155:10'},
   {'id': 'eip155:5000'},
   {'id': 'eip155:314'},
   {'id': 'eip155:412346'},
   {'id': 'eip155:1101'},
   {'id': 'eip155:1516'},
   {'id': 'eip155:80085'},
   {'id': 'eip155:148'},
   {'id': 'eip155:4'},
   {'id': 'eip155:4202'},
   {'id': 'eip155:10143'},
   {'id': 'eip155:420'},
   {'id': 'eip155:88'},
   {'id': 'eip155:1625'},
   {'id': 'eip155:7001'},
   {'

In [ ]:
chains = chains_data['data']['chains']
chains = [c['id'] for c in chains]
chains

['eip155:69',
 'eip155:46',
 'eip155:300',
 'eip155:1480',
 'eip155:534353',
 'eip155:421613',
 'eip155:69000',
 'eip155:84531',
 'eip155:42161',
 'eip155:14800',
 'eip155:89',
 'eip155:5',
 'eip155:59140',
 'eip155:314159',
 'eip155:80001',
 'eip155:7560',
 'eip155:130',
 'eip155:42',
 'eip155:44',
 'eip155:369',
 'eip155:133',
 'eip155:255',
 'eip155:43113',
 'eip155:10',
 'eip155:5000',
 'eip155:314',
 'eip155:412346',
 'eip155:1101',
 'eip155:1516',
 'eip155:80085',
 'eip155:148',
 'eip155:4',
 'eip155:4202',
 'eip155:10143',
 'eip155:420',
 'eip155:88',
 'eip155:1625',
 'eip155:7001',
 'eip155:11155111',
 'eip155:6900',
 'eip155:1380012617',
 'eip155:943',
 'eip155:56',
 'eip155:84532',
 'eip155:13371',
 'eip155:7887',
 'eip155:42220',
 'eip155:59144',
 'eip155:97',
 'eip155:21000001',
 'eip155:1700',
 'eip155:21000000',
 'eip155:545',
 'eip155:43114',
 'eip155:324',
 'eip155:81457',
 'eip155:30',
 'eip155:1301',
 'eip155:1442',
 'eip155:11155420',
 'eip155:534351',
 'eip155:8822'

In [ ]:
#save chains to drive as csv
import pandas as pd

df = pd.DataFrame(chains, columns=['chain_id'])
df.to_csv(drive_path + 'chains.csv', index=False)

# Organizations on chains

In [ ]:
unique_organizations = {}

In [ ]:
import time

def fetch_organizations(chain_id, retries=3):
    for attempt in range(retries):
        variables = {
            "input": {
                "filters": {
                    "chainId": chain_id
                }
            }
        }

        payload = json.dumps({
            "query": query,
            "variables": variables
        })

        response = requests.post(url, headers=headers, data=payload)

        if response.status_code == 200:
            return response.json()

        elif response.status_code == 429:
            wait_time = (2 ** attempt) * 1.5  # Exponential backoff
            print(f"Rate limited on chain {chain_id}, retrying in {wait_time:.1f} seconds...")
            time.sleep(wait_time)

        else:
            print(f"Request failed for chain {chain_id} with status code {response.status_code}")
            print(response.text)
            break

    return None


In [ ]:
query = """
query Organizations($input: OrganizationsInput!) {
  organizations(input: $input) {
    nodes {
      ... on Organization {
        id
        name
        chainIds
        governorIds
        slug
      }
    }
  }
}
"""

for chain_id in chains:
    result = fetch_organizations(chain_id)
    if result:
        organizations = result['data']['organizations']['nodes']
        for org in organizations:
            unique_organizations[org['id']] = org
    time.sleep(1.5)

In [ ]:
len(unique_organizations)

633

# All Organizations

In [ ]:
import requests
import time

query = """
query Organizations($input: OrganizationsInput!) {
  organizations(input: $input) {
    nodes {
      ... on Organization {
        id
        name
        chainIds
        governorIds
        slug
        contracts {
          created_at
          updated_at
        }
      }
    }
    pageInfo {
      lastCursor
    }
  }
}
"""

all_organizations = {}
after_cursor = None
has_next_page = True
delay_seconds = 2  # Set your desired delay here

while has_next_page:
    variables = {
        "input": {
            "page": {
                "limit": 20,  # Hard limit
                "afterCursor": after_cursor
            }
        }
    }

    payload = {
        "query": query,
        "variables": variables
    }

    response = requests.post(url, headers=headers, json=payload)
    data = response.json()

    if 'errors' in data:
        print("Error fetching data:", data['errors'])
        break

    organizations = data['data']['organizations']['nodes']

    if not organizations:
        print("No more organizations to fetch.")
        break

    for org in organizations:
        all_organizations[org['id']] = org

    print(f"Fetched {len(all_organizations)} organizations so far...")

    after_cursor = data['data']['organizations']['pageInfo']['lastCursor']

    if has_next_page:
        print(f"Waiting {delay_seconds} seconds before next request...")
        time.sleep(delay_seconds)

print(f"Total organizations fetched: {len(all_organizations)}")

Fetched 20 organizations so far...
Waiting 2 seconds before next request...
Fetched 40 organizations so far...
Waiting 2 seconds before next request...
Fetched 60 organizations so far...
Waiting 2 seconds before next request...
Fetched 80 organizations so far...
Waiting 2 seconds before next request...
Fetched 100 organizations so far...
Waiting 2 seconds before next request...
Fetched 120 organizations so far...
Waiting 2 seconds before next request...
Fetched 140 organizations so far...
Waiting 2 seconds before next request...
Fetched 160 organizations so far...
Waiting 2 seconds before next request...
Fetched 180 organizations so far...
Waiting 2 seconds before next request...
Fetched 200 organizations so far...
Waiting 2 seconds before next request...
Fetched 220 organizations so far...
Waiting 2 seconds before next request...
Fetched 240 organizations so far...
Waiting 2 seconds before next request...
Fetched 260 organizations so far...
Waiting 2 seconds before next request...
Fet

In [ ]:
import pandas as pd

organizations_df= pd.DataFrame(all_organizations.values())
organizations_df.head()

,id,name,chainIds,governorIds,slug,contracts
0,2206072049795859998,dEaDpOeM,[],[],deadpoem,[]
1,2206072049804248607,Cool,[],[],cool,[]
2,2206072049804248608,dropall,[],[],ac6fbeab-59d1-4087-ba31-fb16f162e9ff,[]
3,2206072049804248609,bigplayerdao,[],[],93cd638a-88e9-4a91-a4cb-962c8b3cfca1,[]
4,2206072049804248610,Storytime DAO,[],[],a2575622-54f9-48e8-a512-6a95c4c14af3,[]


In [ ]:
import json

# After fetching all_organizations
with open('organizations.json', 'w', encoding='utf-8') as f:
    json.dump(all_organizations, f, ensure_ascii=False, indent=4)

print("Data saved as organizations.json")


Data saved as organizations.json


In [ ]:
import json

# Load data from the JSON file
with open('organizations.json', 'r', encoding='utf-8') as f:
    all_organizations = json.load(f)

print(f"Loaded {len(all_organizations)} organizations from the file.")

Loaded 7566 organizations from the file.


# Governors

In [ ]:
import time

# Governors Query
governor_query = """
query Governors($input: GovernorsInput!) {
  governors(input: $input) {
    nodes {
      ... on Governor{
        platform_deployment_id: id
        name
        chainId
        slug
      }
    }
  }
}
"""

governors_list = []

organization_ids_path = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/all_organizations.csv"

# Read the CSV and get all organization IDs
df = pd.read_csv(organization_ids_path)
organization_ids = df['id'].dropna().unique()

for org_id in organization_ids:
    variables = {
        "input": {
            "filters": {
                "organizationId": str(org_id) # Convert org_id to string
            }
        }
    }

    payload = json.dumps({
        "query": governor_query,
        "variables": variables
    })

    response = requests.post(url, headers=headers, data=payload)

    if response.status_code == 200:
      data = response.json()
      governors = data['data']['governors']['nodes']

      if len(governors) > 0:  # Only add if there are governors
          governors_list.extend(governors)
          print(f"Fetched {len(governors)} governors for organization {org_id}")


    elif response.status_code == 429:
        print(f"Rate limited while fetching governors for organization {org_id}, waiting 3 seconds...")
        time.sleep(3)  # Backoff
        continue  # Retry the next iteration

    else:
        print(f"Request failed for organization {org_id} with status code {response.status_code}")
        print(response.text)

    time.sleep(1.5)  # Gentle delay between requests

Streaming output truncated to the last 5000 lines.
No governors found for organization 2206072050114626605
No governors found for organization 2206072050114626606
Fetched 1 governors for organization 2206072050114626607
No governors found for organization 2206072050114626608
Fetched 1 governors for organization 2206072050114626609
Fetched 1 governors for organization 2206072050114626610
No governors found for organization 2206072050114626611
No governors found for organization 2206072050114626612
No governors found for organization 2206072050114626613
Fetched 1 governors for organization 2206072050114626614
Fetched 1 governors for organization 2206072050114626615
No governors found for organization 2206072050114626616
No governors found for organization 2206072050114626617
Fetched 1 governors for organization 2206072050114626618
No governors found for organization 2206072050114626619
Fetched 1 governors for organization 2206072050114626620
Fetched 1 governors for organization 220607205

KeyError: 'id'

In [ ]:
# Optional: remove duplicate governors by id
all_governors = {g['platform_deployment_id']: g for g in governors_list}

print(f"Total governors found: {len(all_governors)}")

Total governors found: 4218


In [ ]:
import json

# After fetching all_organizations
with open('governors.json', 'w', encoding='utf-8') as f:
    json.dump(all_governors, f, ensure_ascii=False, indent=4)

print("Data saved as governors.json")


Data saved as governors.json


In [ ]:
import json

# Load data from the JSON file
with open('governors.json', 'r', encoding='utf-8') as f:
    all_governors = json.load(f)

print(f"Loaded {len(all_governors)} governors from the file.")

In [ ]:
import pandas as pd

# If unique_governors is a dict like {id1: {...}, id2: {...}, ...}
df = pd.DataFrame(all_governors.values())

# Show first few rows
df.head()

,platform_deployment_id,name,chainId,slug
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,eip155:5,rebuild3-pre-alpha
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,eip155:420,more-optimistic-than-ever
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,eip155:10,mydao-2
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,eip155:80001,maxkczf8tmskxeml
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,eip155:80001,idao


In [ ]:
# * `platform`
df['platform'] = 'tally'
# * `platform_id` no chg
# * `name` (may be nan) no chg
# * `website` (may be nan)
df['website'] = None
# * `additional` (may contain website, social, etc. info, may be nan)
df['additional'] = None
# * `votes_count` (the number of unique votes in the deployment)
# TKTK
# * `proposals_count` (the number of proposals in the deployment)
# TKTK
df.head()

,platform_deployment_id,name,chainId,slug,platform,website,additional
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,eip155:5,rebuild3-pre-alpha,tally,None,None
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,eip155:420,more-optimistic-than-ever,tally,None,None
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,eip155:10,mydao-2,tally,None,None
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,eip155:80001,maxkczf8tmskxeml,tally,None,None
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,eip155:80001,idao,tally,None,None


In [ ]:
# Rename 'id' to 'platform_id'
df = df.rename(columns={'id': 'platform_id'})

# Drop 'chainId' and 'slug' columns
df = df.drop(columns=['chainId', 'slug'])

# Reorder columns (if not already in order)
df = df[['platform_deployment_id', 'name', 'platform', 'website', 'additional']]

# Show the cleaned DataFrame
df.head()


,platform_deployment_id,name,platform,website,additional
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,tally,None,None
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,tally,None,None
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,tally,None,None
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,tally,None,None
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,tally,None,None


In [ ]:
# Save DataFrame to CSV
df.to_csv(drive_path + '/governors.csv', index=False)


In [ ]:
df = pd.read_csv('governors.csv')
df.head()

,platform_id,name,platform,website,additional
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,tally,NaN,NaN
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,tally,NaN,NaN
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,tally,NaN,NaN
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,tally,NaN,NaN
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,tally,NaN,NaN


# Proposals

In [ ]:
import json
import requests
import time

# GraphQL Query
proposals_query = """
query Proposals($input: ProposalsInput!) {
  proposals(input: $input) {
    nodes {
      ... on Proposal {
        platform_deployment_id: governor {
          id
        }
        proposal_id: id
        author: proposer {
            id
        }
        date: start {
          ... on Block { # If 'start' is a Block
            timestamp
          }
          ... on BlocklessTimestamp { # If 'start' is a BlocklessTimestamp
            timestamp
          }
        }
        voteStats {
            type
            votesCount
        }
      }
    }
    pageInfo {
      lastCursor
    }
  }
}
"""

def fetch_proposals(governor_id):
    proposals_list = []
    has_next_page = True
    after_cursor = None

    while has_next_page:
        variables = {
            "input": {
                "filters": {
                    "governorId": governor_id
                },
                "page": {
                    "limit": 20,
                    "afterCursor": after_cursor
                }
            }
        }

        payload = json.dumps({
            "query": proposals_query,
            "variables": variables
        })

        response = requests.post(url, headers=headers, data=payload)

        if response.status_code == 200:
            data = response.json()

            if 'errors' in data:
                print(f"Error fetching proposals for governor {governor_id}:", data['errors'])
                break

            if 'data' in data and 'proposals' in data['data'] and 'nodes' in data['data']['proposals']:
                proposals = data['data']['proposals']['nodes']
                proposals_list.extend(proposals)

                if len(proposals) == 0:
                    print(f"No more proposals to fetch for governor {governor_id}")
                    break

                page_info = data['data']['proposals']['pageInfo']
                has_next_page = page_info['lastCursor'] is not None
                after_cursor = page_info['lastCursor']

            else:
                print(f"No proposals found for governor {governor_id}")
                break

            print(f"Fetched {len(proposals)} proposals for governor {governor_id}")

        elif response.status_code == 429:
            print("Rate limited, retrying after 60 seconds...")
            time.sleep(60)
            continue
        else:
            print(f"Request failed with status code {response.status_code}")
            print(response.text)
            break

        time.sleep(1.5)

    return proposals_list

In [ ]:
all_proposals = []

governor_ids = df['platform_id'].dropna().unique().tolist()

for governor_id in governor_ids:
    proposals = fetch_proposals(governor_id)
    all_proposals.extend(proposals)

print(f"Total proposals fetched: {len(all_proposals)}")


Streaming output truncated to the last 5000 lines.
Fetched 1 proposals for governor eip155:137:0x8a3cE624a14aBAAEac8eB70B1A25Cf2f6D4cF36D
Error fetching proposals for governor eip155:137:0x8a3cE624a14aBAAEac8eB70B1A25Cf2f6D4cF36D: [{'message': 'failed to fetch proposals', 'path': ['proposals'], 'extensions': {'code': 13, 'status': {'code': 13, 'message': 'failed to fetch proposals'}}}]
Fetched 1 proposals for governor eip155:137:0xdA87675cE28481A2108dB3d9f692BA8A132e7d01
Rate limited, retrying after 5 seconds...
Error fetching proposals for governor eip155:137:0xdA87675cE28481A2108dB3d9f692BA8A132e7d01: [{'message': 'failed to fetch proposals', 'path': ['proposals'], 'extensions': {'code': 13, 'status': {'code': 13, 'message': 'failed to fetch proposals'}}}]
Fetched 1 proposals for governor eip155:137:0xa43833a51Df6Bc38a5E916C9b69E8ba146875d05
Error fetching proposals for governor eip155:137:0xa43833a51Df6Bc38a5E916C9b69E8ba146875d05: [{'message': 'failed to fetch proposals', 'path': [

In [ ]:
if all_proposals:
    df_proposals = pd.DataFrame(all_proposals)
    df_proposals.to_csv('all_proposals.csv', index=False)
    print("✅ Proposals saved to all_proposals.csv")
else:
    print("⚠️ No proposals found.")

NameError: name 'all_proposals' is not defined

In [ ]:
import pandas as pd

proposals_df = pd.read_csv('all_proposals.csv')
proposals_df = proposals_df[['platform_deployment_id', 'proposal_id']]
proposals_df.head()

,platform_deployment_id,proposal_id
0,{'id': 'eip155:5:0x16449fC3C87b5f4d50BDECB3897...,1999508035835266922
1,{'id': 'eip155:5:0x16449fC3C87b5f4d50BDECB3897...,1999468580705404776
2,{'id': 'eip155:5:0x16449fC3C87b5f4d50BDECB3897...,1994440886435448568
3,{'id': 'eip155:5:0x16449fC3C87b5f4d50BDECB3897...,1994436090710198007
4,{'id': 'eip155:5:0x16449fC3C87b5f4d50BDECB3897...,1994424155826554614


In [ ]:
# rename platform_deployment_id to platform_id
proposals_df = proposals_df.rename(columns={'platform_deployment_id': 'platform_id'})
# group by platform_id and count the number of proposals
proposals_df = proposals_df.groupby('platform_id').count().reset_index()
# rename proposal_id to proposals_count
proposals_df = proposals_df.rename(columns={'proposal_id': 'proposals_count'})
proposals_df.head()

,platform_id,proposals_count
0,{'id': 'eip155:100:0x1f33c126Ad9286c75e7cdfA12...,6
1,{'id': 'eip155:100:0x211F9a4c77E1c98f372Ab0A9d...,3
2,{'id': 'eip155:100:0x4A5327347f077E72d2AaB19F6...,32
3,{'id': 'eip155:100:0x52FC3716321fD8a74793D7E4D...,2
4,{'id': 'eip155:100:0x530Ff2dAED410cA7D70C25f18...,28


In [ ]:
# join
df = df.merge(proposals_df, on='platform_id', how='left', validate='one_to_one')
df.head()

,platform_id,name,platform,website,additional,proposals_count_x,proposals_count_y
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,tally,NaN,NaN,0,NaN
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,tally,NaN,NaN,0,NaN
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,tally,NaN,NaN,0,NaN
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,tally,NaN,NaN,0,NaN
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,tally,NaN,NaN,0,NaN


In [ ]:
# how many non-null proposals_count values are there?
df['proposals_count'].notnull().sum()

KeyError: 'proposals_count'

In [ ]:
# how many null proposals_count values are there?
df['proposals_count'].isnull().sum()

np.int64(4213)

In [ ]:
# fill nulls with 0
df['proposals_count'] = df['proposals_count'].fillna(0)
# make col an int
df['proposals_count'] = df['proposals_count'].astype(int)
df.head()

,platform_id,name,platform,website,additional,proposals_count
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,tally,NaN,NaN,0
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,tally,NaN,NaN,0
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,tally,NaN,NaN,0
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,tally,NaN,NaN,0
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,tally,NaN,NaN,0


In [ ]:
#length of df
len(df)

4213

In [ ]:
df.to_csv('tally_governors_with_proposals_count.csv', index=False)

In [ ]:
import pandas as pd

#csv to df
df = pd.read_csv('tally_governors_with_proposals_count.csv')
df.head()

,platform_id,name,platform,website,additional,proposals_count
0,eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...,ReBuild3 Pre-Alpha,tally,NaN,NaN,0
1,eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...,More Optimistic than Ever,tally,NaN,NaN,0
2,eip155:10:0x2eC637fA69DC102a8F0793632220c37955...,mydao,tally,NaN,NaN,0
3,eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...,maxkczf8tmskxeml,tally,NaN,NaN,0
4,eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...,iDAO,tally,NaN,NaN,0


# Votes

In [ ]:
import pandas as pd
#read  failed proposals and make it df
proposals_df = pd.read_csv('failed_proposal_ids.csv')
proposals_df.head()

,proposal_id
0,2172874406064293829
1,2172874406508890065
2,2140695164249704397
3,2252371325479815118
4,2506359862263285709


In [ ]:
# Extract unique proposal IDs
proposal_ids = proposals_df['proposal_id'].unique().tolist()
len(proposal_ids)

12489

In [ ]:
#proposal ids unique
proposal_ids = list(set(proposal_ids))
len(proposal_ids)

12489

In [ ]:
def split_proposals_by_count(proposal_ids, group_size=200):
    return [proposal_ids[i:i + group_size] for i in range(0, len(proposal_ids), group_size)]

In [ ]:
# GraphQL query
query = """
query Votes($input: VotesInput!) {
  votes(input: $input) {
    nodes {
      ... on OnchainVote {
        id
        voter { id }
        reason
        proposal { id governor { id } }
        type
        amount
        block {
          timestamp
        }
      }
    }
    pageInfo {
      lastCursor
    }
  }
}
"""

In [ ]:
def log_message(message):
    # with log_lock:
    with open(log_file, "a") as log:
        log.write(f"{time.strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")
    print(message)

In [ ]:
import os

tally_votes_folder=drive_path+"tally_votes"
os.makedirs(tally_votes_folder, exist_ok=True)

In [ ]:
def fetch_votes_with_retries(group_index, proposal_id, after_cursor):
    attempt = 0
    backoff_time = 10  # Initial backoff
    votes = []

    while attempt < max_retries:
        try:
            variables = {
                "input": {
                    "filters": {"proposalId": str(proposal_id)},
                    "page": {"limit": 20, "afterCursor": after_cursor}
                }
            }

            payload = {"query": query, "variables": variables}
            response = requests.post(url, headers=headers, json=payload)

            if response.status_code == 429:
                log_message(f"[Group {group_index}] Rate limit hit for proposal {proposal_id}. Backing off for {backoff_time} seconds...")
                time.sleep(backoff_time)
                backoff_time = min(backoff_time * 2, 600)
                attempt += 1
                continue

            if response.status_code != 200:
                log_message(f"[Group {group_index}] HTTP error {response.status_code} for proposal {proposal_id}")
                return False, None, None

            if not response.content.strip():
                log_message(f"[Group {group_index}] Empty response for proposal {proposal_id}")
                return False, None, None

            data = response.json()

            if 'errors' in data:
                error_messages = data['errors']
                log_message(f"[Group {group_index}] Error fetching data for proposal {proposal_id}: {error_messages}")

                # Handle unrecoverable error code 13
                unrecoverable = any(
                    error.get('extensions', {}).get('code') == 13 for error in error_messages
                )

                if unrecoverable:
                    log_message(f"[Group {group_index}] Code 13 detected for proposal {proposal_id}. Skipping to next proposal.")
                    return True, [], None  # Mark as success and no votes to process

                time.sleep(backoff_time * 5)
                backoff_time = min(backoff_time * 2, 600)
                attempt += 1
                continue

            votes = data['data']['votes']['nodes']
            new_cursor = data['data']['votes']['pageInfo']['lastCursor']

            return True, votes, new_cursor  # Success case

        except requests.exceptions.RequestException as e:
            log_message(f"[Group {group_index}] Request error for proposal {proposal_id}: {e}")
            attempt += 1
            time.sleep(backoff_time)
            backoff_time = min(backoff_time * 2, 600)

        except Exception as e:
            log_message(f"[Group {group_index}] Unexpected error for proposal {proposal_id}: {e}")
            attempt += 1
            time.sleep(backoff_time)
            backoff_time = min(backoff_time * 2, 600)

    return False, None, None  # If all attempts failed

In [ ]:
def process_proposal_group(group_index, proposal_group):
    all_votes = []
    failed_proposals = []

    # save_folder = f"{tally_votes_folder}/votes_batches/failed/group_{group_index}"
    # Inside process_proposal_group:
    save_folder = f"{tally_votes_folder}/votes_batches/failed/group_0_recovery"

    os.makedirs(save_folder, exist_ok=True)

    # Detect last batch count based on existing files
    existing_batches = [
        int(re.search(r'votes_batch_(\d+)\.csv', f).group(1))
        for f in os.listdir(save_folder)
        if re.match(r'votes_batch_\d+\.csv', f)
    ]
    batch_count = max(existing_batches, default=0) + 1

    checkpoint_file = f"{save_folder}/progress.json"
    failed_file = f"{save_folder}/failed_proposals.json"

    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint_data = json.load(f)
            start_index = checkpoint_data.get('last_index', 0)
    else:
        start_index = 0

    if os.path.exists(failed_file):
        with open(failed_file, 'r') as f:
            failed_proposals = json.load(f)

    for idx in range(start_index, len(proposal_group)):
        proposal_id = proposal_group[idx]
        after_cursor = None
        log_message(f"[Group {group_index}] Starting proposal {proposal_id} (Index: {idx})")

        while True:
            success, votes, after_cursor = fetch_votes_with_retries(group_index, proposal_id, after_cursor)

            if not success:
                log_message(f"[Group {group_index}] Failed to fetch proposal {proposal_id} after {max_retries} attempts.")
                failed_proposals.append(proposal_id)
                break

            if votes:
                all_votes.extend(votes)
                log_message(f"[Group {group_index}] Fetched votes for proposal {proposal_id}, total votes in memory: {len(all_votes)}")
                time.sleep(delay_seconds)

                # 💾 Save immediately if votes exceed 50,000
                if len(all_votes) >= 50000:
                    batch_filename = os.path.join(save_folder, f"votes_batch_{batch_count}.csv")
                    df = pd.json_normalize(all_votes)
                    df.to_csv(batch_filename, index=False)
                    log_message(f"[Group {group_index}] Saved {len(all_votes)} votes to {batch_filename} (limit hit)")
                    all_votes = []
                    batch_count += 1

            else:
                log_message(f"[Group {group_index}] No more votes for proposal {proposal_id}")
                break

        # ✅ Save at batch boundaries or end of group
        if (idx + 1) % batch_size == 0 or (idx + 1) == len(proposal_group):
            if all_votes:  # Avoid saving empty files
                batch_filename = os.path.join(save_folder, f"votes_batch_{batch_count}.csv")
                df = pd.json_normalize(all_votes)
                df.to_csv(batch_filename, index=False)
                log_message(f"[Group {group_index}] Saved {len(all_votes)} votes to {batch_filename}")
                all_votes = []
                batch_count += 1

            with open(checkpoint_file, 'w') as f:
                json.dump({'last_index': idx + 1}, f)

            with open(failed_file, 'w') as f:
                json.dump(failed_proposals, f, ensure_ascii=False, indent=4)

    log_message(f"[Group {group_index}] Finished processing all proposals.")
    return f"Group {group_index} done"

In [ ]:
import requests
import time
import json
import os
import pandas as pd
import re
# from concurrent.futures import ThreadPoolExecutor
# from threading import Lock

# Configurations
delay_seconds = 5
max_retries = 10
rate_limit_delay = 120
batch_size = 200
# max_workers = 4

# log_lock = Lock()  # For thread-safe logging
log_file = tally_votes_folder + "/votes_log.txt"

proposal_groups = split_proposals_by_count(proposal_ids, group_size=1000)

# with ThreadPoolExecutor(max_workers=max_workers) as executor:
#     futures = [executor.submit(process_proposal_group, count, group) for count, group in enumerate(proposal_groups)]

#     results = [future.result() for future in futures]

results = []

for count, group in enumerate(proposal_groups):
    result = process_proposal_group(count, group)
    print(f"{count} groups has worked")
    results.append(result)

log_message("All groups completed.")
log_message(f"Results: {results}")

[Group 0] Finished processing all proposals.
0 groups has worked
[Group 1] Finished processing all proposals.
1 groups has worked
[Group 2] Finished processing all proposals.
2 groups has worked
[Group 3] Finished processing all proposals.
3 groups has worked
[Group 4] Finished processing all proposals.
4 groups has worked
[Group 5] Finished processing all proposals.
5 groups has worked
[Group 6] Finished processing all proposals.
6 groups has worked
[Group 7] Finished processing all proposals.
7 groups has worked
[Group 8] Finished processing all proposals.
8 groups has worked
[Group 9] Finished processing all proposals.
9 groups has worked
[Group 10] Finished processing all proposals.
10 groups has worked
[Group 11] Finished processing all proposals.
11 groups has worked
[Group 12] Starting proposal 2326957155833349975 (Index: 0)
[Group 12] Fetched votes for proposal 2326957155833349975, total votes in memory: 2
[Group 12] Error fetching data for proposal 2326957155833349975: [{'mess

In [ ]:
def process_proposal_group(group_index, proposal_group):
    all_votes = []
    failed_proposals = []
    batch_count = 1

    save_folder = f"{tally_votes_folder}/votes_batches/failed/group_{group_index}"
    os.makedirs(save_folder, exist_ok=True)

    checkpoint_file = f"{save_folder}/progress.json"
    failed_file = f"{save_folder}/failed_proposals.json"

    # Load checkpoint
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            checkpoint_data = json.load(f)
            start_index = checkpoint_data.get('last_index', 0)
    else:
        start_index = 0

    if os.path.exists(failed_file):
        with open(failed_file, 'r') as f:
            failed_proposals = json.load(f)

    for idx in range(start_index, len(proposal_group)):
        proposal_id = proposal_group[idx]
        after_cursor = None

        while True:
            variables = {
                "input": {
                    "filters": {
                        "proposalId": str(proposal_id)
                    },
                    "page": {
                        "limit": 20,
                        "afterCursor": after_cursor
                    }
                }
            }

            payload = {
                "query": query,
                "variables": variables
            }

            attempt = 0
            success = False
            backoff_time = 10  # Start with 5 seconds for exponential backoff

            while attempt < max_retries:
                try:
                    response = requests.post(url, headers=headers, json=payload)

                    if response.status_code == 429:
                        log_message(f"[Group {group_index}] Rate limit hit for proposal {proposal_id}. Backing off for {backoff_time} seconds...")
                        time.sleep(backoff_time)
                        backoff_time = min(backoff_time * 2, 600)  # Cap backoff at 5 minutes
                        attempt += 1
                        continue

                    if response.status_code != 200:
                        log_message(f"[Group {group_index}] HTTP error {response.status_code} for proposal {proposal_id}")
                        break

                    if not response.content.strip():
                        log_message(f"[Group {group_index}] Empty response for proposal {proposal_id}")
                        break

                    data = response.json()

                    if 'errors' in data:
                        error_messages = data['errors']
                        log_message(f"[Group {group_index}] Error fetching data for proposal {proposal_id}: {error_messages}")

                        unrecoverable = False

                        for error in error_messages:
                            error_code = error.get('extensions', {}).get('code')
                            if error_code == 13:
                                time.sleep(backoff_time * 5)
                                backoff_time = min(backoff_time * 2, 600)
                                log_message(f"[Group {group_index}] Code 13 detected for proposal {proposal_id}. Marking as success and moving on.")
                                unrecoverable = True
                                break  # No need to retry, move to next proposal

                        if unrecoverable:
                            success = True  # Mark as success to skip retries
                            break  # Exit the while True loop for this proposal

                        # If not code 13, perform retries
                        time.sleep(backoff_time * 5)
                        backoff_time = min(backoff_time * 2, 600)
                        attempt += 1
                        continue

                    votes = data['data']['votes']['nodes']

                    if not votes:
                        log_message(f"[Group {group_index}] No more votes for proposal {proposal_id}")
                        success = True
                        break

                    #proposal index
                    log_message(f"[Group {group_index}] Proposal {proposal_id} index: {idx}")
                    all_votes.extend(votes)

                    after_cursor = data['data']['votes']['pageInfo']['lastCursor']
                    log_message(f"[Group {group_index}] Fetched votes for proposal {proposal_id}, total votes in memory: {len(all_votes)}")

                    time.sleep(delay_seconds)
                    success = True
                    break

                except requests.exceptions.RequestException as e:
                    log_message(f"[Group {group_index}] Request error for proposal {proposal_id}: {e}")
                    attempt += 1
                    time.sleep(backoff_time)
                    backoff_time = min(backoff_time * 2, 600)

                except Exception as e:
                    log_message(f"[Group {group_index}] Unexpected error for proposal {proposal_id}: {e}")
                    attempt += 1
                    time.sleep(backoff_time)
                    backoff_time = min(backoff_time * 2, 600)

            if not success:
                log_message(f"[Group {group_index}] Failed to fetch proposal {proposal_id} after {max_retries} attempts.")
                failed_proposals.append(proposal_id)
                break

            if not votes:
                break

        if (idx + 1) % batch_size == 0 or (idx + 1) == len(proposal_group):
            batch_filename = os.path.join(save_folder, f"votes_batch_{batch_count}.csv")
            df = pd.json_normalize(all_votes)
            df.to_csv(batch_filename, index=False)
            log_message(f"[Group {group_index}] Saved {len(all_votes)} votes to {batch_filename}")
            all_votes = []
            batch_count += 1

            with open(checkpoint_file, 'w') as f:
                json.dump({'last_index': idx + 1}, f)

            with open(failed_file, 'w') as f:
                json.dump(failed_proposals, f, ensure_ascii=False, indent=4)

    log_message(f"[Group {group_index}] Finished processing all proposals.")
    return f"Group {group_index} done"


### 🗳️ Proposal Vote Recovery Script

This script recovers on-chain voting data for failed proposals using a GraphQL API. It was designed to retry requests with exponential backoff and save results in manageable batches.

#### 🔧 Use Case

If any proposal data was accidentally overwritten (e.g., in `group_0`), this tool allows selective recovery by filtering out only the proposals **before a specific proposal ID**.

#### ✅ Features

* Retry logic for API errors & rate limits
* Saves votes in `votes_batch_X.csv` files
* Tracks progress with checkpoints
* Logs activity in `votes_log.txt`

#### 💡 Recovery Example

To recover only proposals before ID `2160770615869441795` in `group_0`, run:

```python
group_0 = proposal_groups[0]
cutoff_id = 2160770615869441795
group_0_before_cutoff = [pid for pid in group_0 if int(pid) < cutoff_id]

# Optional: clear previous checkpoint
os.remove(f"{tally_votes_folder}/votes_batches/failed/group_0/progress.json")

# Rerun vote collection
process_proposal_group(0, group_0_before_cutoff)
```

In [ ]:
import re
import time

proposal_groups = split_proposals_by_count(proposal_ids, group_size=1000)
group_0 = proposal_groups[0]  # assuming this is the first 1000 proposals
cutoff_id = 2160770615869441795

# Filter proposals that are less than the cutoff
group_0_before_cutoff = [pid for pid in group_0 if int(pid) < cutoff_id]

#get_cutoff_index
cutoff_index = [i for i, pid in enumerate(proposal_ids) if int(pid) == cutoff_id][0]
print(f"Cutoff index: {cutoff_index}")

# #check if  {tally_votes_folder}/votes_batches/failed/group_0/ exists
# if os.path.exists(f"{tally_votes_folder}/votes_batches/failed/group_0"):
#   if os.path.exists(f"{tally_votes_folder}/votes_batches/failed/group_0/progress.json"):
#     os.remove(f"{tally_votes_folder}/votes_batches/failed/group_0/progress.json")
#   if os.path.exists(f"{tally_votes_folder}/votes_batches/failed/group_0/failed_proposals.json"):
#     os.remove(f"{tally_votes_folder}/votes_batches/failed/group_0/failed_proposals.json")

log_file = f"{tally_votes_folder}/votes_log.txt"
delay_seconds = 5
max_retries = 10
rate_limit_delay = 120
batch_size = 200

# process_proposal_group(0, group_0_before_cutoff)

# results = []

# for count, group in enumerate(proposal_groups):
#     result = process_proposal_group(count, group)
#     print(f"{count} groups has worked")
#     results.append(result)

# log_message("All groups completed.")
# log_message(f"Results: {results}")

499
[Group 0] Finished processing all proposals.
All groups completed.
Results: []


In [ ]:
#sum all votes from all csv and get len
all_votes = []
failed_proposal_ids= []
for folder in os.listdir(tally_votes_folder + "/votes_batches"):
    folder_path = os.path.join(tally_votes_folder + "/votes_batches", folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                all_votes.append(df)
                print(f"Loaded {file_path}")
                print(f"Total votes so far: {len(all_votes)}")
            #add list from failed proposals.json
            if file.endswith("failed_proposals.json"):
                file_path = os.path.join(folder_path, file)
                with open(file_path, 'r') as f:
                    failed_proposal_ids.extend(json.load(f))
                    print(f"Loaded {file_path}")
                    print(f"Total failed proposals so far: {len(failed_proposal_ids)}")
all_votes = pd.concat(all_votes)
len(all_votes)

Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/group_1/failed_proposals.json
Total failed proposals so far: 652
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/group_1/votes_batch_1.csv
Total votes so far: 1
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/group_3/votes_batch_1.csv
Total votes so far: 2
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/group_3/failed_proposals.json
Total failed proposals so far: 1274
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/group_0/votes_batch_1.csv
Total votes so far: 3
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/group_0/failed_proposals.json
Total failed proposals so far: 1926
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/

740922

In [ ]:
#sum all votes from all csv and get len
all_failed_votes = []
failed_proposal_ids= []
for folder in os.listdir(tally_votes_folder + "/votes_batches/failed"):
    folder_path = os.path.join(tally_votes_folder + "/votes_batches/failed", folder)
    if os.path.isdir(folder_path):
        for file in os.listdir(folder_path):
            if file.endswith(".csv"):
                file_path = os.path.join(folder_path, file)
                df = pd.read_csv(file_path)
                all_failed_votes.append(df)
                print(f"Loaded {file_path}")
                print(f"Total votes so far: {len(all_failed_votes)}")
            #add list from failed proposals.json
            if file.endswith("failed_proposals.json"):
                file_path = os.path.join(folder_path, file)
                with open(file_path, 'r') as f:
                    failed_proposal_ids.extend(json.load(f))
                    print(f"Loaded {file_path}")
                    print(f"Total failed proposals so far: {len(failed_proposal_ids)}")
all_failed_votes = pd.concat(all_failed_votes)
len(all_failed_votes)

Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/failed/group_0/votes_batch_3.csv
Total votes so far: 1
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/failed/group_0/votes_batch_1.csv
Total votes so far: 2
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/failed/group_0/failed_proposals.json
Total failed proposals so far: 0
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/failed/group_0/votes_batch_2.csv
Total votes so far: 3
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/failed/group_1/votes_batch_1.csv
Total votes so far: 4
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally//tally_votes/votes_batches/failed/group_1/votes_batch_2.csv
Total votes so far: 5
Loaded /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/ta

1335434

In [ ]:
#make csv for failed proposal list
failed_proposal_ids = pd.DataFrame(failed_proposal_ids, columns=['proposal_id'])
failed_proposal_ids.to_csv("failed_proposal_ids.csv", index=False)

In [ ]:
import shutil

source = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/tally_votes/votes_batches"
destination = "/content/votes_batches"

shutil.copytree(source, destination)

'/content/votes_batches'

In [ ]:
# !rm -rf votes_batches

In [ ]:
# now for votes
votes_df = pd.read_csv('../../votes/tally/tally_votes.csv')
votes_df = votes_df[['platform_deployment_id', 'voter',]]
votes_df.head()

In [ ]:
# group by platform_deployment_id and count the number of voters
votes_df = votes_df.groupby('platform_deployment_id').count().reset_index()
# rename voter to votes_count
votes_df = votes_df.rename(columns={'voter': 'votes_count'})
# rename platform_deployment_id to platform_id
votes_df = votes_df.rename(columns={'platform_deployment_id': 'platform_id'})
votes_df.head()

In [ ]:
# join
df = df.merge(votes_df, on='platform_id', how='left', validate='one_to_one')
df.head()

In [ ]:
# how many nulls vs non nulls in votes_count?
df['votes_count'].isnull().sum(), df['votes_count'].notnull().sum()

In [ ]:
# fill nulls with 0
df['votes_count'] = df['votes_count'].fillna(0)
# make col an int
df['votes_count'] = df['votes_count'].astype(int)
df.head()

In [ ]:
# reorder cols to put votes_count before proposals_count
df = df[['platform', 'platform_id', 'name', 'website', 'additional', 'votes_count', 'proposals_count']]
df.head()

In [ ]:
# save to csv
df.to_csv('tally_deployments.csv', index=False)

NameError: name 'df' is not defined

# Validation of dataset

In [ ]:
import pandas as pd

allgovernors_df=pd.read_csv(drive_path + "all_governors.csv")
deployments_df=pd.read_csv("/content/deployments.csv")

filtered_deployments = deployments_df[(deployments_df['platform'] == 'tally') & (deployments_df['proposals_count'] > 0)]

# Count the deployment ids
deployment_ids_count = len(filtered_deployments)

# Find the deployment ids that are not in the allgovernors id column
missing_ids = filtered_deployments[~filtered_deployments['deployment_id'].isin(allgovernors_df['id'])]

# Display the result
print(f"Count of deployment ids: {deployment_ids_count}")
print(f"Missing deployment ids: {missing_ids['deployment_id'].tolist()}")
len(missing_ids)

Count of deployment ids: 608
Missing deployment ids: ['2dab661d-99b2-5d7b-9404-64afc0b0c747', '2c613c17-1c56-5809-9195-3614e5cc6ca4', 'e35c6732-ee75-5dc0-87cb-5758c12a9910', '2bf281ee-12ad-5d52-99f8-3fc9cd8ae9f8', '73c10c98-7499-53b1-8643-c15d8b5dc5b7', '195dff68-95c4-5ab3-9f26-6bf7beb9c67a', 'b9fe8b04-17b0-5819-8148-1fcf146966d5', 'b4e87533-5707-549c-9983-ac86703a701a', 'ce297fbe-9b37-5341-8d8a-93f2c37a5175', 'c9c682a5-1866-5ba1-80e1-9ef226850759', '9d550003-ea37-5c87-af9a-8d1c4a59809c', '866f25de-a897-515f-ab17-c9ed518616ab', 'f395dff0-b882-5396-a7df-1c37c80b8b22', '3d0efc6d-9f79-50b5-ae2d-23ff3e72ffaf', 'a90784b6-fca7-57dd-a518-fdeefdef481e', '8eb11518-10d7-55e4-bc63-eb70d0374773', '53db4af0-3980-519b-8c65-d1d712a7c6e6', '3270e1ab-3abb-5416-83c7-23bdb56589f7', 'b8e34d8b-e5db-574b-bb09-a3b004654e98', '30f6c6a5-42d2-5c64-bae7-6389e6ec9475', '63bbf6a0-cfe3-5a31-be95-d99584499c6a', 'e80f7ffa-e6db-5902-b8c3-e2d6d5e3565b', '2bb3239b-d9bb-5fe8-9e83-9d01ff6319e7', '862f8be4-fd52-567f-bba9-6

608

# Dataset Buillding

In [ ]:
import os
import pandas as pd

# Path to the main folder
drive_path = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally"
votes_dir = os.path.join(drive_path, "tally_votes")

# List to hold all DataFrames
df_list = []

# Recursively walk through the tally_votes directory
for root, _, files in os.walk(votes_dir):
    for file in files:
        if file.startswith("votes_batch_") and file.endswith(".csv"):
            file_path = os.path.join(root, file)
            try:
                df = pd.read_csv(file_path)
                df_list.append(df)
            except Exception as e:
                print(f"Failed to read {file_path}: {e}")

# Combine all CSVs into one DataFrame
combined_df = pd.concat(df_list, ignore_index=True)

# Remove duplicates based on 'id'
combined_df.drop_duplicates(subset="id", inplace=True)

# Select and rename required columns
final_df = combined_df[[
    "id",
    "proposal.id",
    "proposal.governor.id",
    "voter.id",
    "block.timestamp"
]].rename(columns={
    "id": "vote_id",
    "proposal.id": "proposal_id",
    "proposal.governor.id": "deployment_id",
    "voter.id": "voter",
    "block.timestamp": "date"
})

# Save the final deduplicated file
output_file = os.path.join(drive_path, "all_votes.csv")
final_df.to_csv(output_file, index=False)

print(f"Saved cleaned CSV to: {output_file}")

Saved cleaned CSV to: /content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/all_votes.csv


In [ ]:
import pandas as pd
import ast

# Load the CSV
drive_csv = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/all_proposals.csv"
df = pd.read_csv(drive_csv)

# Parse stringified dictionaries/lists
def parse(x):
    try:
        return ast.literal_eval(x)
    except:
        return None

df['platform_deployment_id'] = df['platform_deployment_id'].apply(parse)
df['date'] = df['date'].apply(parse)
df['voteStats'] = df['voteStats'].apply(parse)

# Extract relevant values
df['deployment_id'] = df['platform_deployment_id'].apply(lambda x: x.get('id') if isinstance(x, dict) else None)
df['date'] = df['date'].apply(lambda x: x.get('timestamp') if isinstance(x, dict) else None)

# Sum vote counts
def sum_votes(votes_list):
    if not isinstance(votes_list, list):
        return 0
    total = 0
    for vote in votes_list:
        try:
            total += int(vote.get('votesCount', 0))
        except:
            pass
    return total

df['votes_count'] = df['voteStats'].apply(sum_votes)

# Keep required columns
df_cleaned = df[['proposal_id', 'deployment_id', 'date', 'votes_count']]

# Drop duplicates by proposal_id
df_cleaned = df_cleaned.drop_duplicates(subset='proposal_id')

# Save cleaned data (optional)
df_cleaned.to_csv("/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/cleaned_proposals.csv", index=False)

print(df_cleaned.head())

           proposal_id                                      deployment_id  \
0  1999508035835266922  eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...   
1  1999468580705404776  eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...   
2  1994440886435448568  eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...   
3  1994436090710198007  eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...   
4  1994424155826554614  eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...   

                   date               votes_count  
0  2023-02-16T19:03:00Z                         0  
1  2023-02-16T17:43:24Z  100000000000000000000000  
2  2023-02-09T19:15:12Z  100000000000000000000000  
3  2023-02-09T19:06:00Z  100000000000000000000000  
4  2023-02-09T18:42:01Z  100000000000000000000000  


In [ ]:
import pandas as pd

# File paths
governors_csv = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/all_governors.csv"
proposals_csv = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/cleaned_proposals.csv"
votes_csv = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/all_votes.csv"

# Load CSVs
df_governors = pd.read_csv(governors_csv).drop_duplicates(subset='id')
df_proposals = pd.read_csv(proposals_csv)
df_votes = pd.read_csv(votes_csv)

# Rename columns and add platform
df_governors.rename(columns={'id': 'deployment_id'}, inplace=True)
df_governors['platform'] = 'tally'

# --- Proposals Count ---
proposal_counts = (
    df_proposals.groupby('deployment_id')
    .size()
    .reset_index(name='proposals_count')
)

# --- Votes Stats ---
vote_stats = (
    df_votes.groupby('deployment_id')
    .agg(
        votes_count=('vote_id', 'count'),
        unique_voters=('voter', pd.Series.nunique)
    )
    .reset_index()
)

# --- Merge Everything ---
df_final = df_governors.merge(proposal_counts, on='deployment_id', how='left')
df_final = df_final.merge(vote_stats, on='deployment_id', how='left')

# Fill NaNs with 0
df_final[['proposals_count', 'votes_count', 'unique_voters']] = df_final[['proposals_count', 'votes_count', 'unique_voters']].fillna(0).astype(int)

# Reorder columns
df_final = df_final[['deployment_id', 'platform', 'name', 'proposals_count', 'unique_voters', 'votes_count']]

# Save result (optional)
df_final.to_csv("/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/deployments.csv", index=False)

# Display result
print(df_final.head())

                                       deployment_id platform  \
0  eip155:5:0x16449fC3C87b5f4d50BDECB389777C3968B...    tally   
1  eip155:420:0x92711600961738b3FE8e66Ca75D5E06D4...    tally   
2  eip155:10:0x2eC637fA69DC102a8F0793632220c37955...    tally   
3  eip155:80001:0x9B78d45BE409FDE95BDc9Bf6f1F2c62...    tally   
4  eip155:80001:0x01bcbEb7f999Ccb9455D70a107A9224...    tally   

                        name  proposals_count  unique_voters  votes_count  
0         ReBuild3 Pre-Alpha                6              1            5  
1  More Optimistic than Ever                1              1            1  
2                      mydao                1              1            1  
3           maxkczf8tmskxeml                2              1            2  
4                       iDAO                2              0            0  


In [ ]:
import pandas as pd

# File paths
proposals_csv = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/cleaned_proposals.csv"
votes_csv = "/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/all_votes.csv"

# Load CSVs
df_proposals = pd.read_csv(proposals_csv)
df_votes = pd.read_csv(votes_csv)

# Convert date columns to datetime
df_proposals['date'] = pd.to_datetime(df_proposals['date'], errors='coerce')
df_votes['date'] = pd.to_datetime(df_votes['date'], errors='coerce')

# Define cutoff date
cutoff = pd.Timestamp("2023-08-01", tz="UTC")

# --- Proposals Count ---
proposals_before = df_proposals[df_proposals['date'] < cutoff].shape[0]
proposals_after = df_proposals[df_proposals['date'] >= cutoff].shape[0]

# --- Votes Count ---
votes_before = df_votes[df_votes['date'] < cutoff].shape[0]
votes_after = df_votes[df_votes['date'] >= cutoff].shape[0]

# --- Deployments Count ---
# We check which deployments have proposals before/after
deployments_before = df_proposals[df_proposals['date'] < cutoff]['deployment_id'].nunique()
deployments_after = df_proposals[df_proposals['date'] >= cutoff]['deployment_id'].nunique()

# --- Output ---
print("📊 Proposal Timeline Summary:")
print(f"Proposals before August 2023: {proposals_before}")
print(f"Proposals from August 2023 onward: {proposals_after}\n")

print("🗳️ Vote Timeline Summary:")
print(f"Votes before August 2023: {votes_before}")
print(f"Votes from August 2023 onward: {votes_after}\n")

print("🏛️ Deployment Timeline Summary:")
print(f"Deployments before August 2023: {deployments_before}")
print(f"Deployments from August 2023 onward: {deployments_after}")

📊 Proposal Timeline Summary:
Proposals before August 2023: 7882
Proposals from August 2023 onward: 12544

🗳️ Vote Timeline Summary:
Votes before August 2023: 510723
Votes from August 2023 onward: 920804

🏛️ Deployment Timeline Summary:
Deployments before August 2023: 1583
Deployments from August 2023 onward: 1386


In [ ]:
import pandas as pd

# Load the final governor summary (deployment-wise stats)
df = pd.read_csv("/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/deployments.csv")

# Define size bins and labels
bins = [2, 10, 100, 1_000, 10_000, 100_000, 1_000_000, 10_000_000]
labels = ['[2,10]', '(10,100]', '(100,1K]', '(1K,10K]', '(10K,100K]', '(100K,1M]', '(1M,10M]']

# Filter: only consider deployments with at least 2 unique voters
filtered_df = df[df["unique_voters"] >= 2].copy()

# Assign size category based on unique_voters
filtered_df["size_category"] = pd.cut(
    filtered_df["unique_voters"],
    bins=bins,
    labels=labels,
    right=True,
    include_lowest=True
)

# Group by size category and platform
table = filtered_df.groupby(["size_category", "platform"]).size().unstack(fill_value=0)

# Add total per row and column
table["Total"] = table.sum(axis=1)
table.loc["Total"] = table.sum(axis=0)

# Reset index for clean display
table = table.reset_index()

# Display settings
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)

# Show or save
print(table)

# Optionally save to CSV
table.to_csv("/content/drive/MyDrive/Egalitarian DAOs/datasets/raw data/tally/tally_size_table.csv", index=False)


platform size_category  tally  Total
0               [2,10]    797    797
1             (10,100]    102    102
2             (100,1K]     40     40
3             (1K,10K]     16     16
4           (10K,100K]      5      5
5            (100K,1M]      2      2
6             (1M,10M]      0      0
7                Total    962    962


/tmp/ipython-input-12-1066793582.py:23: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  table = filtered_df.groupby(["size_category", "platform"]).size().unstack(fill_value=0)
